## 수치미분

**파이토치로하는 자동미분**

**Tensor**

- 넘파이에서 제공하는 다차원 어레이

- 데이터를 텐서 자료형에 넣어야 파이토치에서 제공하는 자동미분 기능 사용 가능

In [ ]:
import torch


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
np.random.seed(0) #랜덤 어레이 생성해서 늘 같은 결과가 나오게 하기 위해서 설정

x = np.random.rand(6).reshape(2,3) #크기가 (2,3)인 넘파이 어레이를 랜덤 생성

x_tensor = torch.tensor(x) #자주 사용하는 몇 가지 방법으로 넘파이 어레이를 텐서로 바꾸기
x_from_numpy = torch.from_numpy(x)
x_Tensor = torch.Tensor(x)
x_as_tensor = torch.as_tensor(x)

print(x, x.dtype)#원 데이터인 넘파이 어레이와 변환된 텐서 출력
print(x_tensor, x_tensor.dtype, x_tensor.requires_grad)
print(x_from_numpy, x_from_numpy.dtype, x_from_numpy.requires_grad)
print(x_Tensor, x_Tensor.dtype, x_Tensor.requires_grad)
print(x_as_tensor, x_as_tensor.dtype, x_as_tensor.requires_grad)

[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]] float64
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]]) torch.float32 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False


In [ ]:
x[0,0] =100 #원 데이터인 넘파이 어레이에서 요소 하나를 바꿨을 때 텐서에 어떤 현상이 일어남

print(x, x.dtype)
print(x_tensor, x_tensor.dtype, x_tensor.requires_grad)
print(x_from_numpy, x_from_numpy.dtype, x_from_numpy.requires_grad)
print(x_Tensor, x_Tensor.dtype, x_Tensor.requires_grad)
print(x_as_tensor, x_as_tensor.dtype, x_as_tensor.requires_grad)

[[100.           0.71518937   0.60276338]
 [  0.54488318   0.4236548    0.64589411]] float64
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]]) torch.float32 False
tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64) torch.float64 False


torch.from_numpy()와 torch.as_tensor()만 값이 함께 바뀌기에 이 두 방식만 넘파이 어레이와 데이터를 공유하고 나머지는 데이터를 복사하여 새롭게 텐서를 만듭니다.



In [ ]:
#requires_grad의 값이 모두 false
#독립변수로 입력받는 함수를 해당 텐서로 자동 미분하려면  True

x_tensor_grad =  torch.tensor(x, requires_grad=True)
print(x_tensor_grad, x_tensor_grad.dtype, x_tensor_grad.requires_grad)

tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64,
       requires_grad=True) torch.float64 True


In [ ]:
x = torch.tensor([1.0], requires_grad=True)
f = (x**2 + 2*x) * torch.log(x)

print(x)
print(f)
print(x.grad)

print(x.grad_fn)
print(f.grad_fn)

tensor([1.], requires_grad=True)
tensor([0.], grad_fn=<MulBackward0>)
None
None


In [ ]:
#x가 마지막 노드인가
#backward()함수는 마지막 노드까지 역전파하면서 미분계수 구한다.
print(x.is_leaf)

True


#### torch.autograd.backward

- torch.autograd

   - 스칼라함수를 자동미분

- torch.autograd.backward

   - 계산 그래프의 끝에서 거꾸로 계산하여 그래프의 시작으로 거슬러 올라간다.

   - 사용자가 만든 변수들에 대한 미분계수를 구해주는 함수

   

In [ ]:
#계산 그래프를 유지하기 위해서 retain_graph = True
#미분계수 : 3
torch.autograd.backward(f,grad_tensors = torch.tensor([1.]), retain_graph=True)
print(x.grad)


tensor([3.])


**torch.autograd.grad**

- 종속변수와 미분할 변수를 모두 명시적으로 지정 후 반환값으로 바로 미분계수를 돌려받는다.

In [ ]:
df = torch.autograd.grad(f,x, retain_graph=True) #f와 미분할 변수 x 넘기기, 여러 개 미분도 가능
print(df)
print(x.grad)

(tensor([3.]),)
tensor([3.])


In [ ]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad= True)
f_xy = (x**2 + 2*x) * torch.log(y)

torch.autograd.backward(f_xy, retain_graph=True)
print(x.grad)
print(y.grad)

df = torch.autograd.grad(f_xy, (x,y), retain_graph=True)
print(df)

tensor([2.7726])
tensor([1.5000])
(tensor([2.7726]), tensor([1.5000]))


### 자동미분 구현

- 구현 방법: 포워드모드 와 리버스 모두

- 목적: 미분하고자 하는 함수를 계산 그래프로 만들어서 미분계수를 계산

- 포워드모드

   - 각 변수에 대해서 편미분계수를 구하려면 그만큼 순전파를 시켜야하므로 비효율적이다.

   - 그러므로, 편미분계수를 구하는 목적이면 사용안함

   
- 리버스모드

   - 순전파, 역전파 각각 한번으로도 편미분계수를 모두 구할 수 있다.

In [ ]:
def times(x,y):
    return x*y, (x,y)

def times_deriv(cache, dout=1):
    return cache[1]*dout, cache[0]*dout

TIMES = {'f': times, 'df': times_deriv}

v, cache = TIMES['f'](2,3)
dx,dy = TIMES['df'](cache)

print("dx={}. dy={}".format(dx,dy))

dx=3. dy=2


In [ ]:
def add(x, y):
    return x+y, (x,y)

def add_deriv(cache, dout=1):
    return dout, dout

ADD = {'f': add, 'df': add_deriv}

def log(x):
    return np.log(x), x

def log_deriv(cache, dout=1):
    return (1/cache)*dout

LOG = {'f': log, 'df': log_deriv}


In [ ]:
x =1.; y = 2.

a, cache_a = TIMES['f'](x,x)
b, cache_b = TIMES['f'](2,x)
c, cache_c = ADD['f'](a,b)
d, cache_d = LOG['f'](y)
z, cache_z = TIMES['f'](c,d)

print("forward pass f(x) = {:.6f}".format(z))

dx = dy =0.
dc, dd =TIMES['df'](cache_z,1)
dy =LOG['df'](cache_d, dd)
da, db = ADD['df'](cache_c, dc)
_, dx_ =TIMES['df'](cache_b, db); dx+=dx_;
dx_, dx__ =  TIMES['df'](cache_a, da); dx+= dx_+dx__;

print('backward pass dx = {:.6f}, dy = {:.6f}'.format(dx,dy))

forward pass f(x) = 2.079442
backward pass dx = 2.772589, dy = 1.500000


In [ ]:
#파이토치로 상류층 미분계수를 2를 주고 미분한 경우

